## Smooth Trend Model

## Install Packages

In [ ]:
!pip install numpyro

【重要】パッケージのインストール完了後に、ランタイムを再起動して下さい！

## Import Packages

In [ ]:
import numpyro
import numpyro.distributions as dist

import jax
import jax.numpy as jnp

import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (8, 5)

In [ ]:
numpyro.set_platform('cpu')
numpyro.set_host_device_count(4)

## Load Data

In [ ]:
data = pd.read_csv('data.csv')

t = data['t']
u = data['u']
y = data['y']

plt.plot(u)
plt.plot(y, '.');

## Define Model & Inference

In [ ]:
y = jnp.array(y)

In [ ]:
def model(y_obs):
    
    num_steps = len(y_obs)
        
    sd_b = numpyro.sample('sd_b', dist.HalfNormal(10))
    
    c = numpyro.sample('c', dist.Normal(10))
    b = numpyro.sample('b', dist.GaussianRandomWalk(scale=sd_b, num_steps=num_steps))
    u = numpyro.deterministic('u', c + jnp.cumsum(b))
        
    sd_y = numpyro.sample('sd_y', dist.HalfNormal(10))
    numpyro.sample('y', dist.Normal(u, sd_y), obs=y_obs)

In [ ]:
nuts = numpyro.infer.NUTS(model, target_accept_prob=0.95)
mcmc = numpyro.infer.MCMC(nuts, num_warmup=1000, num_samples=1000, num_chains=4)

mcmc.run(jax.random.PRNGKey(1), y)

trace = mcmc.get_samples()

In [ ]:
idata = az.from_numpyro(mcmc)

In [ ]:
az.summary(idata, var_names=['c', 'sd_b', 'sd_y'])

In [ ]:
az.plot_trace(idata, var_names=['c', 'sd_b', 'sd_y'])
plt.gcf().subplots_adjust(wspace=0.5, hspace=0.5)

## Check Estimated Trend

In [ ]:
u_sampled = trace['u']

In [ ]:
mu = jnp.mean(u_sampled, 0)
pi = jnp.percentile(u_sampled, jnp.array([5, 95]), 0)

In [ ]:
plt.plot(t, y, '.', color='C1')

plt.plot(t, mu, '-.')
plt.fill_between(t, pi[0, :], pi[1, :], alpha=0.3);